In [6]:
import pandas as pd
from sample_weight_audit import weighted_repeated_fit_equivalence_test
import threadpoolctl

threadpoolctl.threadpool_limits(limits=1)
import warnings
warnings.filterwarnings("ignore")

from sklearn import utils
from inspect import signature

ests = utils.all_estimators(type_filter='transformer')

df = []

for est in ests:
    est = est[1]
    
    if 'random_state' in signature(est).parameters and 'sample_weight' in signature(est.fit).parameters:
        ## record in dataframe name of estimator, and all pvalues and minimum pvalue and average pvalue
        try:
            df.append(paired_test(est, n_features = 10, test='welch', max_seed=20,
                    train_size = 300, n_samples_per_cv_group=500,
                    rep_test_size = 5, max_repeats=3,
                    correct_threshold=True, equal_var=False,issparse=True))
        except ValueError:
            print(est, "with different random state led to the same predictions")
        except:
            print(est, 'does not accept sparse data')
        
df = pd.DataFrame(df)
df.sort_values(by=['min_p_value'])



  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:00<00:00, 187.88it/s]


Finished looping till the maximum random state, 20 for estimator <class 'sklearn.cluster._bisect_k_means.BisectingKMeans'> in ---- 0.13335299491882324 s---
Minimum p-values:  0.9999999999999841
<class 'sklearn.preprocessing._discretization.KBinsDiscretizer'> does not accept sparse data


100%|██████████| 20/20 [00:00<00:00, 218.75it/s]


Finished looping till the maximum random state, 20 for estimator <class 'sklearn.cluster._kmeans.KMeans'> in ---- 0.11328411102294922 s---
Minimum p-values:  0.9999999999999918


100%|██████████| 20/20 [00:00<00:00, 105.83it/s]


Finished looping till the maximum random state, 20 for estimator <class 'sklearn.cluster._kmeans.MiniBatchKMeans'> in ---- 0.21426105499267578 s---
Minimum p-values:  0.02300241927604689


100%|██████████| 20/20 [00:00<00:00, 45.64it/s]

Finished looping till the maximum random state, 20 for estimator <class 'sklearn.ensemble._forest.RandomTreesEmbedding'> in ---- 0.45311522483825684 s---
Minimum p-values:  0.732616574772321


,Name,p_values,min_p_value,avg_p_value
2,MiniBatchKMeans,"[0.20744416989537867, 0.3635330869820961, 0.13...",0.023002,0.334900
3,RandomTreesEmbedding,"[0.732616574772321, 0.9723868479485198, 0.8877...",0.732617,0.865353
0,BisectingKMeans,"[0.9999999999999993, 0.9999999999999953, 0.999...",1.000000,1.000000
1,KMeans,"[0.9999999999999998, 0.999999999999994, 1.0, 1...",1.000000,1.000000
